In [1]:
import pandas as pd 
import numpy as np

In [2]:
dataset = pd.read_csv('ratings.csv')
dataset.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movieTitles = pd.read_csv('movies.csv')
movieTitles.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
dataset = pd.merge(dataset, movieTitles, on='movieId')
dataset.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [5]:
movieMatrix = dataset.pivot_table(index='userId', columns='movieId', values='rating')
movieMatrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
R = movieMatrix.transpose().fillna(movieMatrix.transpose().mean())
R = R.transpose()
R.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,4.366379,4.000000,4.366379,4.366379,4.000000,4.366379,4.366379,4.366379,4.366379,...,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379
2,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,...,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276
3,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,...,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897
4,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,...,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556
5,4.000000,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,...,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364


In [27]:
R = np.array(R)
R.shape

(610, 9724)

In [28]:
numUsers, numItems = R.shape

In [29]:
K = 400

In [30]:
# Gives r-rank approximation

def lowRankApprox(A=None, r=1):
    """
    Computes an r-rank approximation of a matrix
    given the component u, s, and v of it's SVD
    Requires: numpy
    """
    A = A.T
    
    SVD = np.linalg.svd(A, full_matrices=False)
    u, s, v = SVD
    Ar = np.zeros((len(u), len(v)))
    for i in range(r):
        Ar += s[i] * np.outer(u.T[i], v[i])
    return Ar.T

In [31]:
where_are_NaNs = np.isnan(movieMatrix)
where_are_NaNs = np.array(where_are_NaNs)
where_are_NaNs

array([[False,  True, False, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [False, False, False, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True]])

In [32]:
for i in range(1000):
    kSVD = lowRankApprox(A=R, r=K)
    error = R - kSVD
    
    for j in range(0, numUsers):
        for k in range(0, numItems):
            if where_are_NaNs[j, k]:
                R[j, k] = kSVD[j, k]
                
    # Calculating norm
    error[where_are_NaNs] = 0
    print("Iteration: ", i, " ---> Frobenius norm: ", np.linalg.norm(error))

Iteration:  0  ---> Frobenius norm:  33.87350255285948
Iteration:  1  ---> Frobenius norm:  19.70461311323114
Iteration:  2  ---> Frobenius norm:  12.279298212087982
Iteration:  3  ---> Frobenius norm:  7.9167504826476485
Iteration:  4  ---> Frobenius norm:  5.259766162196243
Iteration:  5  ---> Frobenius norm:  3.6052718863068955
Iteration:  6  ---> Frobenius norm:  2.5497463028885416
Iteration:  7  ---> Frobenius norm:  1.8567253451274535
Iteration:  8  ---> Frobenius norm:  1.3869107572722519
Iteration:  9  ---> Frobenius norm:  1.057986828086343
Iteration:  10  ---> Frobenius norm:  0.8208547575260208
Iteration:  11  ---> Frobenius norm:  0.645638740631605
Iteration:  12  ---> Frobenius norm:  0.5136012547223591
Iteration:  13  ---> Frobenius norm:  0.4125528553223405
Iteration:  14  ---> Frobenius norm:  0.3342627972949905
Iteration:  15  ---> Frobenius norm:  0.2729851463025319
Iteration:  16  ---> Frobenius norm:  0.2245986620262996
Iteration:  17  ---> Frobenius norm:  0.186085

KeyboardInterrupt: 

In [34]:
finalFilledMatrix = lowRankApprox(A=R, r=K)
finalFilledMatrix

array([[3.99999942, 4.33808685, 4.00000002, ..., 4.36645293, 4.36645293,
        4.36689099],
       [4.10473577, 4.04951199, 4.03568229, ..., 3.94652212, 3.94652212,
        3.95063123],
       [2.43343814, 2.43761088, 2.44168205, ..., 2.43591402, 2.43591402,
        2.43700856],
       ...,
       [2.50000006, 1.99999994, 1.9999999 , ..., 3.13424564, 3.13424564,
        3.13355347],
       [2.99997517, 3.0152489 , 3.46552693, ..., 3.27204121, 3.27204121,
        3.2707129 ],
       [4.99999988, 3.69353573, 3.69583252, ..., 3.68866214, 3.68866214,
        3.6883463 ]])